## Python basics
<br>

In [ ]:
# the basis: variable assignment
corpus = "This is a very tiny corpus. But at least this tiny corpus has a second sentence."

In [ ]:
# check the content of a variable
print(corpus)

In [ ]:
# in jupyter notebooks that's enough to print a variable - if it's the last row of a cell
corpus

In [ ]:
# check the data type of any Python object (everything is an object in Python)
type(corpus)

In [ ]:
# objects / data types have their own methods
tokens = corpus.split()
tokens

In [ ]:
# split() returns a list (as indicated by the square brackets)
type(tokens)

In [ ]:
# iterable/sequential data types support selection and operations based on index positions
tokens[1]

In [ ]:
# operations can be nested and/or concatenated
[tokens[0], type(tokens[0])]

In [ ]:
# strings are iterable too
corpus[0]

In [ ]:
# use the "replace" method to improve our tokenization on whitespace
corpus = corpus.replace(".", " .")
corpus

In [ ]:
# now the punctuation gets splitted correctly
tokens = corpus.split()
tokens

In [ ]:
# the built-in method len() calculates the length of iterable data types
token_count = len(tokens)
token_count

In [ ]:
# token_count is an integer object
type(token_count)

In [ ]:
# now we know that our corpus consists of 18 tokens and 82 characters
char_count = len(corpus)
char_count

In [ ]:
# with a for-loop we can access every item in a list or in other iterable data types
for token in tokens:
    print("token:\t" + token) # strings can be concatenated
    #print(f"token:\t{token}") # the f-string syntax does the trick as well

In [ ]:
# let's get the sentences of our corpus
sentences = corpus.split(".")
sentences

In [ ]:
# we have two tasks: get rid of the empty string and
# append the punctuation back to the sentences
sentences_stripped = []
for sentence in sentences:
    sentence = sentence.strip()
    # with conditions we can check if a statement is true or false
    if not sentence == "":
        sentence += " ."
        sentences_stripped.append(sentence)

sentences_stripped

In [ ]:
# dicts provide a mapping between keys and values
token_lengths = {}
type(token_lengths)

In [ ]:
# we can assign values to keys with dict_variable[key] = value
for token in tokens:
    token_lengths[token] = len(token)

token_lengths

In [ ]:
# let's write a simple counter for the token frequencies of our corpus
counter = {}
for token in tokens:
    # check if we have already seen this token
    if token in counter:
        counter[token] += 1
    # if not (we see this token for the first time):
    else:
        counter[token] = 1

counter

In [ ]:
# often programming tasks can be solved in more than one way
counter = {}
for token in tokens:
    if not token in counter:
        counter[token] = 0
    counter[token] += 1

counter

In [ ]:
# for many problems code already exists, which can be imported
# from built-in or external Python modules
from collections import Counter

In [ ]:
# well, that's shorter...
Counter(tokens)

In [ ]:
type(Counter(tokens))

In [ ]:
# we can use the most_common() method of Counter
# to sort our token frequencies
Counter(tokens).most_common()

<br><br>
## spaCy
<br>

In [ ]:
# first we have to import the previously installed library
import spacy

In [ ]:
# let's create a tiny corpus
spacy_corpus = "Mary Lou McDonald grew up in a republican household in Dublin to a backdrop of the Troubles in Northern Ireland. \"My family's connections with the IRA would have been in the 1920s,\" she says."

In [ ]:
# befor spaCy can annotate anything, we have to load a trained statistical model
nlp = spacy.load("en_core_web_sm")

In [ ]:
# nlp() runs the default model annotation pipeline on our example corpus
# 'doc' now contains the annotated spaCy data object
doc = nlp(spacy_corpus)

In [ ]:
# count the tokens of our corpus document
len(doc)

In [ ]:
# spaCy docs are composed of Token objects
print(type(doc[0]))
doc[0]

In [ ]:
# Token objects have different attributes containing their linguistic annotations
for token in doc:
    print(token.text)
    print(token.pos_)
    print(token.pos_, token.tag_)
    print(token.lemma_)
    print(token.morph)
    print(token.ent_iob_, token.ent_type_, "\n")

In [ ]:
spacy.explain("NORP")

In [ ]:
for sentence in doc.sents:
    print(sentence)

In [ ]:
for entity in doc.ents:
    print(entity)

In [ ]:
type(doc.ents[0])

In [ ]:
vrt = "<text>\n"
for sentence in doc.sents:
    vrt += "<s>\n"
    entity = False
    for token in sentence:
        if entity == False and token.ent_iob_ == "B":
            entity = True
            vrt += f"<entity type=\"{token.ent_type_}\">\n"
        elif entity == True and not token.ent_iob_ == "I":
            entity = False
            vrt += "</entity type>\n"
        vrt += "\t".join((token.text, token.pos_, token.lemma_, str(token.morph), token.ent_type_, token.ent_iob_)) + "\n"
    vrt += "</s>\n"
vrt += "</text>\n"

print(vrt)

In [ ]:
vrt = "<text>\n"
for sentence in doc.sents:
    vrt += "<sentence>\n"
    for token in sentence:
        vrt += token.text + "\n"
    vrt += "</sentence>\n"
vrt += "</text>\n"

print(vrt)

In [ ]:
vrt = "<person>\nMary\nLou\nMcDonald\n</person>"
print(vrt)

<br><br>
## data formats
<br>

In [ ]:
import json
import csv

In [ ]:
tweets = []

with open("tweets.jsonl", encoding="utf8") as jsonl:
    for tweet in jsonl:
        tweets.append(json.loads(tweet))

In [ ]:
with open("tweets.csv", "w", encoding="utf8", newline="") as csvfile:
    fieldnames = ["author_id", "conversation_id", "created_at", "id", "in_reply_to_user_id", "like_count", "quote_count", "reply_count", "retweet_count", "referenced_tweet_ids", "reference_types", "text"]
    writer = csv.DictWriter(csvfile, dialect="excel", delimiter=";", fieldnames=fieldnames, extrasaction="ignore")
    writer.writeheader()
    
    for tweet in tweets:        
        tweet_fields = {
            "author_id": tweet["author_id"],
            "conversation_id": tweet["conversation_id"],
            "created_at": tweet["created_at"],
            "id": tweet["id"],
            "in_reply_to_user_id": tweet.get("in_reply_to_user_id"),
            "like_count": tweet.get("public_metrics", {}).get("like_count"),
            "quote_count": tweet.get("public_metrics", {}).get("quote_count"),
            "reply_count": tweet.get("public_metrics", {}).get("reply_count"),
            "retweet_count": tweet.get("public_metrics", {}).get("retweet_count"),
            "referenced_tweet_ids": ", ".join([referenced.get("id") for referenced in tweet.get("referenced_tweets", {})]),
            "reference_types": ", ".join([referenced.get("type") for referenced in tweet.get("referenced_tweets", {})]),
            "text": tweet["text"].replace("\n", " ")
        }
        writer.writerow(tweet_fields)

<br><br>
## trafilatura
<br>

In [ ]:
import trafilatura

In [ ]:
url = "https://europa.eu/citizens-initiative-forum/blog/europeans-safe-connections-call-stronger-regulation-wireless-internet-schools_en"
downloaded = trafilatura.fetch_url(url)

In [ ]:
result = trafilatura.extract(downloaded)
result

In [ ]:
result = trafilatura.extract(
    downloaded,
    output_format="xml",
    url=url,
    #include_comments=True,
    #include_formatting=True,
    #include_links=True,
    #include_images=True,
    #include_tables=True,
    #favor_precision=True,
    #favor_recall=True,
    #target_language="en"
)

In [ ]:
print(result)

In [ ]:
from xml.dom import minidom

In [ ]:
print(minidom.parseString(result).toprettyxml(indent="    "))

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))

In [ ]:
print(trafilatura.extract(downloaded, output_format="csv"))

In [ ]:
print(trafilatura.extract(downloaded, output_format="json"))

In [ ]:
from trafilatura.spider import focused_crawler

In [ ]:
homepage = "https://europa.eu/citizens-initiative-forum/blog_en"

In [ ]:
# starting a crawl
to_visit, known_urls = focused_crawler(homepage, max_seen_urls=10, max_known_urls=100000)

In [ ]:
list(to_visit)

In [ ]:
sorted(known_urls)

In [ ]:
# resuming a crawl
to_visit, known_urls = focused_crawler(homepage, max_seen_urls=10, max_known_urls=100000, todo=to_visit, known_links=known_urls)

In [ ]:
# filter the crawl links
sorted([url for url in known_urls if url.startswith("https://europa.eu/citizens-initiative-forum/blog/")])

<br><br>
## Data analysis and visualisation with Python
<br>

In [ ]:
from collections import Counter
from pathlib import Path
import xml.etree.ElementTree as ET

import spacy
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
corpus = Path("nexis_corpus").rglob("*.xml")

articles = []
for newspaper in corpus:
    xml_tree = ET.parse(str(newspaper))
    doc_elements = xml_tree.findall("document")
    
    for doc in doc_elements:
        articles.append(
            {
                "newspaper": doc.find("metadata/source").text,
                "publication_type": doc.find("metadata/publication_type").text,
                "distribution": doc.find("metadata/distribution").text,
                "year": doc.find("metadata/year").text,
                "text": " ".join([paragraph.strip() for paragraph in doc.find("text").itertext()]).strip(),
            }
        )

articles[:3]

In [ ]:
texts = [article["text"] for article in articles]

In [ ]:
nlp = spacy.load("en_core_web_sm")
docs = list(nlp.pipe(texts))

for article, doc in zip(articles, docs):
    article["tagged"] = doc

In [ ]:
tokens = [token for doc in docs for token in doc if not token.is_space]
tokens[:10]

In [ ]:
lemmas = [token.lemma_ for token in tokens]
lemmas[:10]

In [ ]:
counter = Counter(lemmas).most_common()

In [ ]:
counter[:20]

In [ ]:
lemmas_sorted = []
counts = []
rank = []
for i, (lemma, count) in enumerate(counter, 1):
    lemmas_sorted.append(lemma)
    counts.append(count)
    rank.append(str(i))

In [ ]:
fig = go.Figure(
    data=go.Scatter(
        x=lemmas_sorted[:50],
        y=counts[:50],
        text=rank,
        hovertemplate = "<i>lemma</i>: <b>%{x}</b><br><i>count</i>: %{y}<br><i>rank</i>: %{text}<extra></extra>",
        #line_shape="spline"
    )
)

fig.update_layout(
    title="Zipf distribution",
    xaxis_title="Lemma (lemmatized by spaCy)",
    yaxis_title="Occurences in corpus",
    template="simple_white"
)

fig.show()

In [ ]:
fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=("Scatter chart", "Bar chart"),
    x_title="Lemma (lemmatized by spaCy)",
    y_title="Occurences in corpus",
)

fig.add_trace(
    go.Scatter(
        x=lemmas_sorted[:30],
        y=counts[:30],
        text=rank,
        hovertemplate = "<i>lemma</i>: <b>%{x}</b><br><i>count</i>: %{y}<br><i>rank</i>: %{text}<extra></extra>"
    ),
    row=1,
    col=1
)

fig.add_trace(
    go.Bar(
        x=lemmas_sorted[:30],
        y=counts[:30],
        text=rank,
        hovertemplate = "<i>lemma</i>: <b>%{x}</b><br><i>count</i>: %{y}<br><i>rank</i>: %{text}<extra></extra>"
    ),
    row=1,
    col=2
)

fig.update_layout(
    title="Zipf distribution",
    template="seaborn",
    showlegend=False
)

fig.layout.annotations[0]["yshift"] = 10
fig.layout.annotations[1]["yshift"] = 10
fig.layout.annotations[2]["yshift"] = -50

fig.show()

In [ ]:
text_token_distribution = {
    "Sunday Mail": {"texts": 0, "tokens": 0},
    "The Guardian": {"texts": 0, "tokens": 0},
    "The Times": {"texts": 0, "tokens": 0},
}

for article in articles:
    newspaper = article["newspaper"]
    text_token_distribution[newspaper]["texts"] += 1
    text_token_distribution[newspaper]["tokens"] += len(article["tagged"])

labels, texts, tokens = zip(*[(newspaper, values["texts"], values["tokens"]) for newspaper, values in text_token_distribution.items()])

text_token_distribution

In [ ]:
colors = ["lightsalmon", "lightsteelblue", "gold"]

fig = make_subplots(
    rows=1,
    cols=2,
    specs=[[{"type" :"domain"}, {"type": "domain"}]]
)

fig.add_trace(
    go.Pie(
        labels=labels,
        values=texts,
        direction="counterclockwise",
        rotation=0,
        sort=True,
        hole=.3
    ),
    row=1,
    col=1
)

fig.add_trace(
    go.Pie(
        labels=labels,
        values=tokens,
        direction="counterclockwise",
        rotation=0,
        sort=True,
        hole=.3
    ),
    row=1,
    col=2
)

fig.update_traces(
    textinfo="label+percent",
    textfont_size=15,
    hovertemplate="<b>%{value}</b><extra></extra>",
    marker=dict(colors=colors, line=dict(color='#000000', width=2.3))
)

fig.update_layout(
    title="Text and token distribution - Pie chart",
    width=1000,
    height=500,
    showlegend=False,
    annotations=[
        dict(text="Texts", x=0.195, y=0.5, font_size=17, showarrow=False),
        dict(text="Tokens", x=0.815, y=0.5, font_size=17, showarrow=False)
    ]
)

fig.show()

In [ ]:
fig = go.Figure(
    data=[
        go.Bar(
            name="Texts",
            x=labels,
            y=[t/sum(texts) for t in texts],
            marker_color="steelblue"
        ),
        go.Bar(
            name="Tokens",
            x=labels,
            y=[t/sum(tokens) for t in tokens],
            marker_color="salmon"
        )
    ]
)

fig.update_traces(
    marker_line_color="rgb(8,48,107)",
    marker_line_width=1.5, opacity=0.6
)

fig.update_layout(
    barmode="group",
    bargroupgap=0.05,
    title="Text and token distribution - Bar chart",
    xaxis_title="Newspaper",
    yaxis_title="Percentage",
    template="ggplot2"
)

fig.show()

In [ ]:
query_distribution = {
    "Sunday Mail": {"hits": 0, "in_texts": 0, "per_text": []},
    "The Guardian": {"hits": 0, "in_texts": 0, "per_text": []},
    "The Times": {"hits": 0, "in_texts": 0, "per_text": []},
}

for article in articles:
    tokens = [token.text for token in article["tagged"]]
    if "London" in tokens:
        per_text = 0
        for token in tokens:
            if token == "London":
                query_distribution[article["newspaper"]]["hits"] += 1
                per_text += 1
        query_distribution[article["newspaper"]]["in_texts"] += 1
        query_distribution[article["newspaper"]]["per_text"].append(per_text)

print(query_distribution)

In [ ]:
colors = ["lightsalmon", "lightsteelblue", "gold"]

fig = make_subplots(
    rows=1,
    cols=3,
    specs=[[{"type" :"domain"}, {"type": "domain"}, {"type": "domain"}]],
    subplot_titles=(
        "Absolute frequency",
        "Relative frequency (instances per million words)",
        "Dispersion (texts with at least one hit)"
    ),
)

fig.add_trace(
    go.Pie(
        labels=labels,
        values=[query_distribution[label]["hits"] for label in labels],
        direction="counterclockwise",
        rotation=0,
        sort=True
    ),
    row=1,
    col=1
)

fig.add_trace(
    go.Pie(
        labels=labels,
        values=[(query_distribution[label]["hits"]/text_token_distribution[label]["tokens"])*1000000 for label in labels],
        direction="counterclockwise",
        rotation=0,
        sort=True
    ),
    row=1,
    col=2
)

fig.add_trace(
    go.Pie(
        labels=labels,
        values=[query_distribution[label]["in_texts"] for label in labels],
        direction="counterclockwise",
        rotation=0,
        sort=True
    ),
    row=1,
    col=3
)

fig.update_traces(
    textinfo="label+percent",
    textfont_size=15,
    hovertemplate="<b>%{value}</b><extra></extra>",
    marker=dict(colors=colors, line=dict(color='#000000', width=2.3))
)

fig.update_layout(
    title="Results for query \"London\" - absolute vs. relative frequency vs. dispersion",
    showlegend=False
)

fig.layout.annotations[2]["yshift"] = -30

fig.show()

In [ ]:
colors = ["lightsalmon", "lightsteelblue", "gold"]

fig = go.Figure()

for label in labels:
    fig.add_trace(
        go.Violin(
            y=query_distribution[label]["per_text"],
            name=label,
            box_visible=True,
            meanline_visible=True
        )
    )

fig.update_layout(
    title="Results for query \"London\" - Dispersion (hits per text)",
    showlegend=False,
    yaxis_title="No. hits per text",
    template="ggplot2"
)

fig.show()